# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [7]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [8]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [9]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [10]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [11]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 105us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 38us/step - loss: nan 

1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 39us/step - loss

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [19]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.fit_transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.fit_transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [20]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [21]:
# Train the model 
normalized_input_model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 113us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 39us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 40

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [22]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [24]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs=150, batch_size=32, validation_data=(X_val, y_val) )

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 177us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 36093030765.9924 - val_mean_squared_error: 36093030765.9924
Epoch 2/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 36093027277.3840 - val_mean_squared_error: 36093027277.3840
Epoch 3/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 36093064834.4335 - val_mean_squared_error: 36093064834.4335
Epoch 4/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 36093013322.9506 - val_mean_squared_error: 36093013322.9506
Epoch 5/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 36093040032.6084 - val_mean_squared_error: 36093040032.6084
Epoch 6/150
1051/1051 

Epoch 45/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0361 - mean_squared_error: 0.0361 - val_loss: 36093026997.0494 - val_mean_squared_error: 36093026997.0494
Epoch 46/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0351 - mean_squared_error: 0.0351 - val_loss: 36093013050.4030 - val_mean_squared_error: 36093013050.4030
Epoch 47/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0345 - mean_squared_error: 0.0345 - val_loss: 36093013104.9125 - val_mean_squared_error: 36093013104.9125
Epoch 48/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0338 - mean_squared_error: 0.0338 - val_loss: 36093021600.6084 - val_mean_squared_error: 36093021600.6084
Epoch 49/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0327 - mean_squared_error: 0.0327 - val_loss: 36093009616.3042 - val_mean_squared_error: 36093009616.3042
Epoch 50/150
1051/1051 [==============================] - 0s 40us

Epoch 89/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0162 - mean_squared_error: 0.0162 - val_loss: 36093019085.3840 - val_mean_squared_error: 36093019085.3840
Epoch 90/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0160 - mean_squared_error: 0.0160 - val_loss: 36093032066.4335 - val_mean_squared_error: 36093032066.4335
Epoch 91/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0160 - mean_squared_error: 0.0160 - val_loss: 36093022683.0114 - val_mean_squared_error: 36093022683.0114
Epoch 92/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0157 - mean_squared_error: 0.0157 - val_loss: 36093025229.3840 - val_mean_squared_error: 36093025229.3840
Epoch 93/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0155 - mean_squared_error: 0.0155 - val_loss: 36093025174.8745 - val_mean_squared_error: 36093025174.8745
Epoch 94/150
1051/1051 [==============================] - 0s 40us

Epoch 133/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0096 - mean_squared_error: 0.0096 - val_loss: 36093023516.2281 - val_mean_squared_error: 36093023516.2281
Epoch 134/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 36093022215.7871 - val_mean_squared_error: 36093022215.7871
Epoch 135/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 36093019918.6008 - val_mean_squared_error: 36093019918.6008
Epoch 136/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 36093019171.0418 - val_mean_squared_error: 36093019171.0418
Epoch 137/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 36093022215.7871 - val_mean_squared_error: 36093022215.7871
Epoch 138/150
1051/1051 [==============================] - 0

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on training data
training_results = normalized_model.evaluate(X_train, y_train_scaled)
print('Training Loss:', training_results[0], 'Training MSE:', training_results[1])

1051/1051 [==============================] - 0s 24us/step
Training Loss: 0.007754646180078855 Training MSE: 0.007754646180078855


- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [35]:
# Evaluate the model on validate data
test_results = normalized_model.evaluate(X_val, y_val_scaled)
print('Test Loss:',test_results[0], 'Test MSE:',test_results[1])

263/263 [==============================] - 0s 32us/step
Test Loss: 0.11214345540610557 Test MSE: 0.11214345540610557


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [36]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26315.914204017823

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [41]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 153us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1872 - val_mean_squared_error: 0.1872
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1567 - val_mean_squared_error: 0.1567
Epoch 3/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1798 - mean_squared_error: 0.1798 - val_loss: 0.1387 - val_mean_squared_error: 0.1387
Epoch 4/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1569 - mean_squared_error: 0.1569 - val_loss: 0.1461 - val_mean_squared_error: 0.1461
Epoch 5/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1320 - val_mean_squared_error: 0.1320
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

1051/1051 [==============================] - 0s 42us/step - loss: 0.0315 - mean_squared_error: 0.0315 - val_loss: 0.1123 - val_mean_squared_error: 0.1123
Epoch 50/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0304 - mean_squared_error: 0.0304 - val_loss: 0.1029 - val_mean_squared_error: 0.1029
Epoch 51/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1074 - val_mean_squared_error: 0.1074
Epoch 52/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 53/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 54/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0282 - mean_squared_error: 0.0282 - val_loss: 0.1118 - val_mean_squared_error: 0.1118
Epoch 55/15

Epoch 98/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0147 - mean_squared_error: 0.0147 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 99/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1095 - val_mean_squared_error: 0.1095
Epoch 100/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 101/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1204 - val_mean_squared_error: 0.1204
Epoch 102/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1187 - val_mean_squared_error: 0.1187
Epoch 103/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1175 - val_mean_squared_error: 0

Epoch 147/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1210 - val_mean_squared_error: 0.1210
Epoch 148/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1237 - val_mean_squared_error: 0.1237
Epoch 149/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1203 - val_mean_squared_error: 0.1203
Epoch 150/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1179 - val_mean_squared_error: 0.1179


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [42]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 27us/step


[0.008186745135242972, 0.008186745135242972]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [43]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.1178771798205013, 0.1178771798205013]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [44]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 160us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1646 - val_mean_squared_error: 0.1646
Epoch 2/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1371 - val_mean_squared_error: 0.1371
Epoch 3/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1226 - val_mean_squared_error: 0.1226
Epoch 4/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1291 - val_mean_squared_error: 0.1291
Epoch 5/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.1166 - val_mean_squared_error: 0.1166
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1197 - mean_squared_error: 0.1197 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1035 - val_mean_squared_error: 0.1035
Epoch 51/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0262 - mean_squared_error: 0.0262 - val_loss: 0.1071 - val_mean_squared_error: 0.1071
Epoch 52/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 53/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0248 - mean_squared_error: 0.0248 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 55/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0245 - mean_squared_error: 0.0245 - val_loss: 0.1088 - val_mean_squared_error: 0.108

Epoch 99/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1074 - val_mean_squared_error: 0.1074
Epoch 100/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 101/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1207 - val_mean_squared_error: 0.1207
Epoch 102/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1174 - val_mean_squared_error: 0.1174
Epoch 103/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1173 - val_mean_squared_error: 0.1173
Epoch 104/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1215 - val_mean_squared_error: 

Epoch 148/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1228 - val_mean_squared_error: 0.1228
Epoch 149/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1181 - val_mean_squared_error: 0.1181
Epoch 150/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1153 - val_mean_squared_error: 0.1153


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [45]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 25us/step


[0.007070233428196806, 0.007070233428196806]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [46]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 38us/step


[0.11533936400660544, 0.11533936400660544]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [47]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 188us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1386 - val_mean_squared_error: 0.1386
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1666 - mean_squared_error: 0.1666 - val_loss: 0.1010 - val_mean_squared_error: 0.1010
Epoch 3/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1242 - mean_squared_error: 0.1242 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 4/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.1296 - val_mean_squared_error: 0.1296
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0888 - mean_squared_error: 0.0888 - val_loss: 0.1242 - val_mean_squared_error: 0.1242
Epoch 6/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0661 - mean_squared_error: 0.0661 - val_lo

Epoch 49/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.0882 - val_mean_squared_error: 0.0882
Epoch 50/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.0937 - val_mean_squared_error: 0.0937
Epoch 51/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0897 - val_mean_squared_error: 0.0897
Epoch 52/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0925 - val_mean_squared_error: 0.0925
Epoch 53/150
1051/1051 [==============================] - ETA: 0s - loss: 0.0130 - mean_squared_error: 0.01 - 0s 42us/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 54/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0083 - mean_squared_error: 0.008

1051/1051 [==============================] - 0s 42us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0905 - val_mean_squared_error: 0.0905
Epoch 98/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0912 - val_mean_squared_error: 0.0912
Epoch 99/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0912 - val_mean_squared_error: 0.0912
Epoch 100/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.1044 - val_mean_squared_error: 0.1044
Epoch 101/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0844 - val_mean_squared_error: 0.0844
Epoch 102/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0839 - val_mean_squared_error: 0.0839
Epoch 10

Epoch 146/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0921 - val_mean_squared_error: 0.0921
Epoch 147/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0925 - val_mean_squared_error: 0.0925
Epoch 148/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0890 - val_mean_squared_error: 0.0890
Epoch 149/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0892 - val_mean_squared_error: 0.0892
Epoch 150/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0898 - val_mean_squared_error: 0.0898


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [48]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 27us/step


[0.0037076253741696037, 0.0037076253741696037]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [49]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.08975526915700263, 0.08975526915700263]

In [52]:
rms_scaled_preds = rmsprop_model.predict(X_val)
rms_preds = ss_y.inverse_transform(rms_scaled_preds)
rmsRMSE = np.sqrt(mean_squared_error(y_val, rms_preds))
print(rmsRMSE)

23542.981302462395


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [53]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 221us/step - loss: 0.3832 - mean_squared_error: 0.3832 - val_loss: 0.1711 - val_mean_squared_error: 0.1711
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1617 - mean_squared_error: 0.1617 - val_loss: 0.1255 - val_mean_squared_error: 0.1255
Epoch 3/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1127 - mean_squared_error: 0.1127 - val_loss: 0.1008 - val_mean_squared_error: 0.1008
Epoch 4/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0863 - mean_squared_error: 0.0863 - val_loss: 0.1154 - val_mean_squared_error: 0.1154
Epoch 5/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1010 - val_mean_squared_error: 0.1010
Epoch 6/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0569 - mean_squared_error: 0.0569 - val_lo

1051/1051 [==============================] - 0s 45us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.1073 - val_mean_squared_error: 0.1073
Epoch 50/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.1139 - val_mean_squared_error: 0.1139
Epoch 51/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 52/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.1091 - val_mean_squared_error: 0.1091
Epoch 53/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.1134 - val_mean_squared_error: 0.1134
Epoch 54/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.1053 - val_mean_squared_error: 0.1053
Epoch 55/15

Epoch 98/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.1000 - val_mean_squared_error: 0.1000
Epoch 99/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.1058 - val_mean_squared_error: 0.1058
Epoch 100/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0967 - val_mean_squared_error: 0.0967
Epoch 101/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.1004 - val_mean_squared_error: 0.1004
Epoch 102/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0948 - val_mean_squared_error: 0.0948
Epoch 103/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.1012 - val_mean_squared_error: 0

1051/1051 [==============================] - 0s 46us/step - loss: 2.9234e-04 - mean_squared_error: 2.9234e-04 - val_loss: 0.0969 - val_mean_squared_error: 0.0969
Epoch 147/150
1051/1051 [==============================] - 0s 47us/step - loss: 4.4154e-04 - mean_squared_error: 4.4154e-04 - val_loss: 0.0919 - val_mean_squared_error: 0.0919
Epoch 148/150
1051/1051 [==============================] - 0s 44us/step - loss: 5.9587e-04 - mean_squared_error: 5.9587e-04 - val_loss: 0.0996 - val_mean_squared_error: 0.0996
Epoch 149/150
1051/1051 [==============================] - 0s 42us/step - loss: 9.3372e-04 - mean_squared_error: 9.3372e-04 - val_loss: 0.0904 - val_mean_squared_error: 0.0904
Epoch 150/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0929 - val_mean_squared_error: 0.0929


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [54]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 27us/step


[0.003578787715175144, 0.003578787715175144]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [55]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.09289853604857459, 0.09289853604857459]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [77]:
# Evaluate the best model on test data
model_list = [adam_model, rmsprop_model, he_model, lecun_model]
y_preds = {}
for model in model_list:
    y_preds[str(model)] = ss_y.inverse_transform(model.predict(X_val))
    y_preds[str(model)] = np.sqrt(mean_squared_error(y_val, y_preds[str(model)]))
# print(y_preds)
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 48us/step


[0.1762399673461914, 0.1762399673461914]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [78]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_preds = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_preds))

32990.10418250082

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.